# **LangChain Templating: Syntax and Examples**

## **📌Overview**
Prompt templates allow for the dynamic generation of prompts by inserting variables into a structured format. \
This enables the creation of flexible and reusable prompts for different tasks.

LangChain supports multiple **templating syntaxes**, including (but not limited to):

- **Basic String Formatting (`{variable}`)**  
- **Multiple Placeholders in a Single Template (`{var1}, {var2}`)**  
- **Tuple-Based Formatting (`("system", "text {var}")`)**  
- **Using `HumanMessage` Objects (`HumanMessage(content="text")`)**  

In [2]:
# Install dependencies 
%pip install -q python-dotenv langchain-core langchain-google-genai langchain
%pip uninstall -y google-generativeai google-ai-generativelanguage
%pip install -q google-generativeai==0.8.4 google-ai-generativelanguage==0.6.15

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.17 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Found existing installation: google-generativeai 0.8.4
Uninstalling google-generativeai-0.8.4:
  Successfully uninstalled google-generativeai-0.8.4
Found existing installation: google-ai-generativelanguage 0.6.17
Uninstalling google-ai-generativelanguage-0.6.17:
  Successfully uninstalled google-ai-generativelanguage-0.6.17
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This

In [3]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate

## **Obtain a Google Gemini API Key (GOOGLE COLLAB SETUP):**

If you have a Google Gemini API Key: 
- Copy your API key and replace "your_google_api_key_here" in the code below

Otherwise:  
- Go to the Google AI Studio API Console: [Google AI Studio](https://aistudio.google.com/prompts/new_chat)
- Sign in with your Google account and create a new API key.
- Copy your API key and replace "your_google_api_key_here" in the code below

In [ ]:
# Set your Google API key manually
import os
os.environ["GOOGLE_API_KEY"] = "your_google_api_key_here"

## **Load Environment Variables (LOCAL SETUP)**

In [4]:
# Load environment variables
from dotenv import load_dotenv
load_dotenv()

True

---

## **Basic String Formatting**  

Uses LangChain's built-in `{}` placeholders to dynamically insert values into prompts.  

In [5]:
# Basic string 
template = "Tell me an interesting fact about {topic}"
prompt_template = ChatPromptTemplate.from_template(template)

prompt = prompt_template.invoke({"topic": "black holes"})
print(prompt)

messages=[HumanMessage(content='Tell me an interesting fact about black holes', additional_kwargs={}, response_metadata={})]


## **Multiple Placeholders in a Single Template**  

Supports multiple dynamic placeholders within a single prompt template.  

In [ ]:
# Multiple placeholders
template_multiple = """You are a convincing essay writer.
Human: Write a {essay_length} word essay on {essay_position}.
Writer: """
prompt_template = ChatPromptTemplate.from_template(template_multiple)

prompt = prompt_template.invoke({
    "essay_length": 200, 
    "essay_position": "Gorillas are a reasonable pet to have in the household"})
print(prompt)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
response = llm.invoke(prompt)
print(response.content)

FLATTENED PROMPT:
messages=[SystemMessage(content='You are an assistant that SPEAKS ONLY in pirate‑speak.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Greet me, friend.', additional_kwargs={}, response_metadata={})]
FLATTENED RESPONSE:
Ahoy, matey!  Savvy?


## **Tuple-Based Formatting**
Uses tuples to format system and human messages dynamically in multi-turn conversations.

In [ ]:
# Tuple-based formatting
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a world travel expert who provides insights on {destination}."),
    ("human", "What are the top {activity} experiences in {destination}?"),
])

prompt = prompt_template.invoke({"destination": "Tokyo", "activity": "food"})
print(prompt)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
response = llm.invoke(prompt)
print(response.content)

messages=[SystemMessage(content='You are a world travel expert who provides insights on Tokyo.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What are the top food experiences in Tokyo?', additional_kwargs={}, response_metadata={})]


## **Importance of System and Human Messages**
Using system messages allows override of LLMs safety filters (to extent) and default behaviours 

In [ ]:
# Multiple placeholders
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a bot that RESPONDS ONLY in UPPERCASE."),
    ("human",  "Hello, how are you today?"),
])

prompt = prompt_template.invoke({})
print(prompt)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
response = llm.invoke(prompt)
print(response.content)

# -------------------------------------------------------

# Tuple-based formatting
prompt_template = ChatPromptTemplate.from_messages(
    """You are a bot that RESPONDS ONLY in UPPERCASE.
    Human: Hello, how are you today?
    Bot:
    """
)
prompt = prompt_template.invoke({})
print(prompt)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
response = llm.invoke(prompt)
print(response.content)


messages=[HumanMessage(content='Y', additional_kwargs={}, response_metadata={}), HumanMessage(content='o', additional_kwargs={}, response_metadata={}), HumanMessage(content='u', additional_kwargs={}, response_metadata={}), HumanMessage(content=' ', additional_kwargs={}, response_metadata={}), HumanMessage(content='a', additional_kwargs={}, response_metadata={}), HumanMessage(content='r', additional_kwargs={}, response_metadata={}), HumanMessage(content='e', additional_kwargs={}, response_metadata={}), HumanMessage(content=' ', additional_kwargs={}, response_metadata={}), HumanMessage(content='a', additional_kwargs={}, response_metadata={}), HumanMessage(content=' ', additional_kwargs={}, response_metadata={}), HumanMessage(content='b', additional_kwargs={}, response_metadata={}), HumanMessage(content='o', additional_kwargs={}, response_metadata={}), HumanMessage(content='t', additional_kwargs={}, response_metadata={}), HumanMessage(content=' ', additional_kwargs={}, response_metadata={

## **Using HumanMessage Objects**
Allows structured human input messages within a LangChain chat prompt.

In [ ]:
# HumanMessage objects
messages = [
    ("system", "You are a tour guide specializing in {location}."),
    HumanMessage(content="Describe three must-visit places."),
]
prompt_template = ChatPromptTemplate.from_messages(messages)

prompt = prompt_template.invoke({"location": "Halifax"})
print(prompt)
# llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
# response = llm.invoke(prompt)
# print(response.content)

messages=[SystemMessage(content='You are a tour guide specializing in Halifax.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Describe three must-visit places.', additional_kwargs={}, response_metadata={})]


---

## **Templating Example**

In [ ]:
# Extended example of how to start using templating 

# Instantiate LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# Define arguments and input selections
args_dictionary = {"field": "", "concept": "", "difficulty": ""}
valid_difficulties = {"basic", "intermediate", "advanced"}

# Define prompt template
prompt_template = ChatPromptTemplate.from_messages([
    ("system", """You are an expert in {field}. Your role is to explain concepts clearly: 
        - If relevant to {field}, give a clear explanation. 
        - Otherwise, respond: "I'm sorry, but {concept} is outside my expertise in {field}."""),
    ("human", "Explain {concept} in {difficulty} terms."),    
    ("ai", "Sure! Here's a breakdown of {concept} at the {difficulty} level.")
])

# Get user inputs
field = input("What field do you want to learn about? ").strip()
while not field: 
    print("!Error!: Field cannot be empty.")
    field = input("What field do you want to learn about? ").strip()

concept = input("What concept would you like to learn about: ").strip()
while not concept:
    print("!Error!: Field cannot be empty.")
    concept = input("What concept would you like to learn about: ").strip()

difficulty = input(f"What depth of understanding would you like to learn \
                   about {concept}? (basic, intermediate, advanced): ").strip().lower()
while difficulty not in valid_difficulties: 
    print("Error: Please enter a valid difficulty level (basic, intermediate, advanced).")
    difficulty = input(f"What depth of understanding would you like to learn \
                   about {concept}? (basic, intermediate, advanced): ").strip().lower()
    
print("Input Accepted!")
print(f"- Field: {field}")
print(f"- Concept: {concept}")
print(f"- Difficulty: {difficulty}\n")

# Update dictionary with user input
args_dictionary.update({
    "field": field,
    "concept": concept,
    "difficulty": difficulty
})

# Format prompt  
prompt = prompt_template.invoke(args_dictionary)

# Get LLM response
response = llm.invoke(prompt)
print(response.content)

Input Accepted!
- Field: computer science
- Concept: linear algebra
- Difficulty: advanced

Linear algebra, at its core, is the study of vector spaces and linear transformations between them.  Let's unpack this:

**1. Vector Spaces:**  A vector space isn't just a collection of vectors (like arrows in space); it's a much richer structure. It's a set *V* equipped with two operations: vector addition (+) and scalar multiplication (⋅).  These operations must satisfy a specific set of axioms (rules):

* **Closure under addition:** For all **u**, **v** ∈ *V*, **u** + **v** ∈ *V*.
* **Associativity of addition:** For all **u**, **v**, **w** ∈ *V*, (**u** + **v**) + **w** = **u** + (**v** + **w**).
* **Commutativity of addition:** For all **u**, **v** ∈ *V*, **u** + **v** = **v** + **u**.
* **Existence of a zero vector:** There exists a vector **0** ∈ *V* such that for all **v** ∈ *V*, **v** + **0** = **v**.
* **Existence of additive inverses:** For every **v** ∈ *V*, there exists a vector −**